In [15]:
import requests, bs4, re, os, time, tqdm, csv, json
import pandas as pd
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import seaborn as sns

keyword = "网络个人信息侵权"

# 读取json并转化成DataFrame
file = f"./data/{keyword}_contents.json"
with open(file, "r", encoding="utf-8") as f:
    contents = json.load(f)
df = pd.DataFrame(contents)
# 修改标题
df.rename(columns={"title": "标题", "case_number": "案号", "court": "法院", "data": "日期", "cause": "案由", "type": "类型", "procedure": "程序", "procedure_explain": "程序说明", "tags": "标签", "opinion": "观点", "verdict": "判决"}, inplace=True)


In [16]:
# 创建一个result文件夹
if not os.path.exists("./result"):
    os.mkdir("./result")

In [17]:
df.head()

,标题,案号,法院,日期,案由,类型,程序,程序说明,标签,观点,判决
0,孙长宝与北京搜狐互联网信息服务有限公司等人格权纠纷一审民事判决书,北京互联网法院,北京互联网法院,2020年09月10日,人格权纠纷,判决书,一审,原告孙某某1与被告北京百度网讯科技有限公司（以下简称百度网讯公司）人格权纠纷一案，本院于20...,"[个人隐私, 网络服务, 追加, 迟延履行, 连带责任, 简易程序, 备案, 免责, 申请人...",本院认为，结合各方当事人的诉辩意见及在案证据，本案的争议焦点为:一、涉案姓名、照片及其关联关...,综上，依照《中华人民共和国民法总则》第一百一十条 、第一百一十一条 ，《中华人民共和国侵权责...
1,吴某、张某、葛某某等侵犯公民个人信息罪一审刑事判决书,滁州市琅琊区人民法院,滁州市琅琊区人民法院,2020年06月03日,侵犯公民个人信息罪,判决书,一审,安徽省滁州市琅琊区人民检察院以琅检一部刑诉[2020]19号起诉书指控被告人吴某某1、张某某...,"[缓刑, 从轻处罚, 取保候审, 并处罚金, 认罪, 如实供述, 证据不足, 自首, 立功,...",本院认为，被告人吴某某1、张某某1、葛某某1、董某某1违反国家规定，非法获取、出售公民个人信...,依照《中华人民共和国刑法》第二百五十三条 之一第一款 、第三款 、第二百八十七条 之一第一款...
2,黄华侵犯公民个人信息罪一审刑事判决书,永泰县人民法院,永泰县人民法院,2021年02月26日,侵犯公民个人信息罪,判决书,一审,永泰县人民检察院以樟检一部刑诉［2021］Z10号起诉书指控被告人黄某某1犯侵犯公民个人信息...,"[缓刑, 从宽处理, 从轻处罚, 取保候审, 并处罚金, 认罪, 如实供述]",本院认为，被告人黄某某1伙同他人违反国家有关规定，向他人出售公民个人信息，情节严重，其行为已...,综上，依照《中华人民共和国刑法》第二百五十三条 之一第一款 、第二十五条 第一款 、第六十一...
3,安思洁、刘平兴隐私权纠纷再审审查与审判监督民事裁定书,四川省高级人民法院,四川省高级人民法院,2019年03月08日,隐私权纠纷,裁定书,再审,再审申请人安某某1因与被申请人刘某某1、王某某1隐私权纠纷一案，不服四川省绵阳市中级人民法院...,,,依照《中华人民共和国民事诉讼法》第二百零四条 第一款 ，《最高人民法院关于适用的解释》第三百...
4,苏州贝尔塔数据技术有限公司与伊日克斯庆一般人格权纠纷二审民事判决书,江苏省苏州市中级人民法院,江苏省苏州市中级人民法院,2020年06月10日,一般人格权纠纷,判决书,二审,上诉人苏州贝尔塔数据技术有限公司（以下简称贝尔塔公司）因与被上诉人伊某某1人格权纠纷一案，不...,"[备案, 网络服务, 工商登记, 转载, 新证据, 公证书, 法定程序, 迟延履行, 加倍支...",本院认为，根据《中华人民共和国民法总则》第一百一十一条 之规定，自然人的个人信息受法律保护。...,本院依照《中华人民共和国民法总则》第一百一十一条 、《中华人民共和国侵权责任法》第二十条 、...


In [18]:
# 转换日期格式 2020年06月10日 -> 2020-06-10
df["日期"] = df["日期"].apply(lambda x: x.replace("年", "-").replace("月", "-").replace("日", ""))
# 改变日期数据类型
df["日期"] = pd.to_datetime(df["日期"])
# 查看数据类型
df.dtypes

标题              object
案号              object
法院              object
日期      datetime64[ns]
案由              object
类型              object
程序              object
程序说明            object
标签              object
观点              object
判决              object
dtype: object

In [19]:
len(df)

2000

In [20]:
# 获取['判决']中所有被《》包裹的内容，放进一个数组
laws = df["判决"].apply(lambda x: re.findall(r"《(.*?)》", x))
# 合并law里面的所有，去掉重复项
laws = list(set([i for j in laws for i in j]))
# 只留下laws中带有“法”字的
laws = [i for i in laws if "法" in i]
# 查看laws
laws
# 保存laws
# with open("./result/laws.txt", "w", encoding="utf-8") as f:
#     f.write("\n".join(laws))

['最高人民法院最高人民检察院关于检察公益诉讼案件适用法律若干问题的解释',
 '关于审理利用信息网络侵害人身权益民事纠纷案件适用法律若干问题的规定',
 '最高人民法院关于审理名誉权案件若干问题的解释',
 '中华人民共和国刑法',
 '最高人民法院关于审理著侵害信息网络传播权民事纠纷案件适用法律若干问题的规定',
 '中华人民共国侵权责任法',
 '最高人民法院关于确认民事侵权精神损害赔偿责任若干问题的解释',
 '关于检察公益诉讼案件适用法律若干问题的解释',
 '关于审理不正当竞争民事案件应用法律若干问题的解释',
 '中华人民共和国婚姻法',
 '最高人民法院、最高人民检察院',
 '中-8-华人民共和国民事诉讼法',
 '中华人民共和国商标法实施条例',
 '中华人民共和国涉外民事关系法律适用法',
 '最高人民法院关于互联网法院审理案件若干问题的规定',
 '最高人民法院关于适用《中华人民共和国行政诉讼法',
 '诉讼费用交纳办法',
 '最高人民法院关于审理利用信息网络侵害人身权益民事纠纷案件适用法律若干问题的定',
 '最高人民法院关于审理消费民事公益诉讼案件适用法律若干问题的解释',
 '最高人民法院关于审理专利纠纷案件适用法律问题的若干规定',
 '最高人民法院最高人民检察院关于办理侵犯公民个人信息刑事案件适用法律若干问题的解释',
 '最高人民法院关于审理侵犯专利权纠纷案件应用法律若干问题的解释',
 '最高人法院关于确定民事侵权精神损害赔偿责任若干问题的解释',
 '最高人民法院关于审理利用信息网络侵害人身权益民事案件适用法律若干问题的规定',
 '中华人民共和国行政诉讼法',
 '中华人共和国民法通则',
 '最高人民法院关于适用《中华人民共和国民事诉讼法（?javascript:SLC（183386,0）?）',
 '《中华人民共和国侵权责任法',
 '最高人民法院关于审理著作权民事纠纷案件适用法律若干问题的解释',
 '最高人民法院关于执行程序中计算迟延履行期间的债务利息适用法律若干问题的解释',
 '最高人民法院关于适用中华人民共和国民事诉讼法',
 '中华人民共和国商标法',
 '中华人民共和国民事诉讼法（2007年修正）',
 '最高人民法院关于审理利用信息网络侵害人身权益民事纠纷案件适用法律若干问题的解释',
 '最高人民法院

In [21]:
# 读取 law_dic.txt 里面存放着所有的法律文书名称，我要用来做词云
with open("./dic/law_dic.txt", "r", encoding="utf-8") as f:
    law_dic = f.read().split("\n")
# 查看law_dic
for law in law_dic:
    print(law)

关于审理名誉权案件若干问题的解释
关于审理利用信息网络侵害人身权益民事纠纷案件适用法律若干问题的定
关于审理侵害信息网络传播权民事纠纷案件适用法律若干问题
关于检察公益诉讼案件适用法律若干问题的解释
中华人民共和国民事诉讼法
关于审理利用信息网络侵害民事权益纠纷案件适用法律若干问题的规定
关于审理行政赔偿案件若干问题的规定
关于检察公益诉讼案件适用法律若干问题的解释
中华人民共国侵权责任法
关于审理不正当竞争民事案件应用法律若干问题的解释
中华人民共和国消费者权益保护法
关于确定民事侵权精神损害陪若干问题的解释
关于审理名誉权若干问题的解答
中华人民共和国道路交通安全法
中华人民共和国刑事诉讼法
中华人共和国民法通则
关于办理侵犯公民个人信息刑事案件适用法律若干问题的解释
关于审理侵害信息网络传播权民事纠纷案件适用法律若干问题的规定
中华人民共和国行政诉讼法
诉讼费用交纳办法
关于审理利用网络侵害人身权益民事纠纷案件适用法律若干问题的规定
中华人民共和国侵权责任法
关于审理名誉权案件若干问题的解答
中华人民共和国涉外民事关系法律适用法
中华人民共和国继承法
关于办理侵犯公民个人信息刑事案件适用法律若干问题的解释
关于审理利用信息网络侵害人身权益民事纠纷案件适用法律问题的规定
中华人民共和国刑事诉讼法
关于适用时间效力的若干规定
关于适用中华人民共和国行政诉讼法若干问题的解释
关于确定民事侵权精神损害赔偿责任若干问题的解释
关于审理侵害信息网络传播权民事纠纷案件适用法律若干问题的规定
关于检察公益诉讼案件适用法律若干问题的解释
关于办理侵犯公民个人信息刑事案件适用法律若干问题的解释
关于适用中华人民共和国民事诉讼法的解释
关于审理利用信息网络侵害人身权益民事纠纷案件适用法律若干的规定
关于审理名誉权案件若干问题的解答
关于适用中华人民共和国民事诉讼法
关于审理利用信息网络侵害人身权益民事纠纷案件适用法律若干问题的规定
关于执行程序中计算迟延履行期间的债务利息适用法律若干问题的解释
关于审理民事案件适用诉讼时效制度若干问题的规定
关于适用诉讼时效制度若干问题的解释
关于审理环境民事公益诉讼案件适用法律若干问题的解释
关于精神损害赔偿的司法解释
中华人民共和国民法典
中华人民共和国反不正当竞争法
中华人民共和国民法通则
关于贯彻执行若干问题的意见
关于确认民事侵权精神损害

In [22]:
# 统计所有 '判决'中 law_dic 里面的词出现的次数，然后做词云
# 用一个字典来存放所有的词和出现的次数
law_dic_count = {}
# 遍历所有的词
for law in law_dic:
    # 统计每个词出现的次数
    law_dic_count[law] = df["判决"].apply(lambda x: x.count(law)).sum()
# 查看law_dic_count
for k, v in law_dic_count.items():
    print(k, v)
# 生成词云
wc = WordCloud(font_path="./fonts/微软雅黑.ttf", background_color="white", width=1000, height=800, max_words=100, max_font_size=150)
wc.generate_from_frequencies(law_dic_count)
plt.figure(figsize=(10, 5), dpi=300)
# 保存
wc.to_file(f"./result/{keyword}_法条词云.png")

# 先对law_dic_count进行降序排序，然后取前10个
law_dic_count = dict(sorted(law_dic_count.items(), key=lambda x: x[1], reverse=True)[:10])
# 字体倾斜显示
# 对law_dic_count绘制横向的bar图
plt.figure(figsize=(15, 7), dpi=300)
plt.xlabel("出现次数")
plt.ylabel("法条")
plt.title(f"{keyword}_法条词频")
plt.barh(list(law_dic_count.keys()), list(law_dic_count.values()))
plt.savefig(f"./result/{keyword}_法条词频.png", dpi=300, bbox_inches='tight')


关于审理名誉权案件若干问题的解释 8
关于审理利用信息网络侵害人身权益民事纠纷案件适用法律若干问题的定 1
关于审理侵害信息网络传播权民事纠纷案件适用法律若干问题 107
关于检察公益诉讼案件适用法律若干问题的解释 31
中华人民共和国民事诉讼法 2292
关于审理利用信息网络侵害民事权益纠纷案件适用法律若干问题的规定 3
关于审理行政赔偿案件若干问题的规定 1
中华人民共国侵权责任法 1
关于审理不正当竞争民事案件应用法律若干问题的解释 5
中华人民共和国消费者权益保护法 25
关于确定民事侵权精神损害陪若干问题的解释 1
关于审理名誉权若干问题的解答 2
中华人民共和国道路交通安全法 1
中华人民共和国刑事诉讼法 56
中华人共和国民法通则 2
关于办理侵犯公民个人信息刑事案件适用法律若干问题的解释 30
关于审理侵害信息网络传播权民事纠纷案件适用法律若干问题的规定 106
中华人民共和国行政诉讼法 17
诉讼费用交纳办法 2
关于审理利用网络侵害人身权益民事纠纷案件适用法律若干问题的规定 2
中华人民共和国侵权责任法 566
关于审理名誉权案件若干问题的解答 41
中华人民共和国涉外民事关系法律适用法 1
中华人民共和国继承法 2
关于审理利用信息网络侵害人身权益民事纠纷案件适用法律问题的规定 1
关于适用时间效力的若干规定 12
关于适用中华人民共和国行政诉讼法若干问题的解释 0
关于确定民事侵权精神损害赔偿责任若干问题的解释 150
关于适用中华人民共和国民事诉讼法的解释 2
关于审理利用信息网络侵害人身权益民事纠纷案件适用法律若干的规定 1
关于适用中华人民共和国民事诉讼法 3
关于审理利用信息网络侵害人身权益民事纠纷案件适用法律若干问题的规定 190
关于执行程序中计算迟延履行期间的债务利息适用法律若干问题的解释 1
关于审理民事案件适用诉讼时效制度若干问题的规定 2
关于适用诉讼时效制度若干问题的解释 4
关于审理环境民事公益诉讼案件适用法律若干问题的解释 1
关于精神损害赔偿的司法解释 1
中华人民共和国民法典 50
中华人民共和国反不正当竞争法 11
中华人民共和国民法通则 189
关于贯彻执行若干问题的意见 8
关于确认民事侵权精神损害赔偿责任若干问题的解释 1
中华人民共和国合同法 35
关于审理利用信息网络侵害人身权利民事纠纷案件适用法律若

<ipython-input-22-42946b36a7d6>:27: UserWarning: Glyph 32593 (\N{CJK UNIFIED IDEOGRAPH-7F51}) missing from current font.
  plt.savefig(f"./result/{keyword}_法条词频.png", dpi=300, bbox_inches='tight')
<ipython-input-22-42946b36a7d6>:27: UserWarning: Glyph 32476 (\N{CJK UNIFIED IDEOGRAPH-7EDC}) missing from current font.
  plt.savefig(f"./result/{keyword}_法条词频.png", dpi=300, bbox_inches='tight')
<ipython-input-22-42946b36a7d6>:27: UserWarning: Glyph 20010 (\N{CJK UNIFIED IDEOGRAPH-4E2A}) missing from current font.
  plt.savefig(f"./result/{keyword}_法条词频.png", dpi=300, bbox_inches='tight')
<ipython-input-22-42946b36a7d6>:27: UserWarning: Glyph 20154 (\N{CJK UNIFIED IDEOGRAPH-4EBA}) missing from current font.
  plt.savefig(f"./result/{keyword}_法条词频.png", dpi=300, bbox_inches='tight')
<ipython-input-22-42946b36a7d6>:27: UserWarning: Glyph 20449 (\N{CJK UNIFIED IDEOGRAPH-4FE1}) missing from current font.
  plt.savefig(f"./result/{keyword}_法条词频.png", dpi=300, bbox_inches='tight')
<ipython-input-

In [23]:
# 对'案由'做词频统计，做词云
# 用一个字典来存放所有的词和出现的次数
cause_count = {}
# 遍历所有的词
for cause in df["案由"]:
    # 统计每个词出现的次数
    cause_count[cause] = df["案由"].apply(lambda x: x.count(cause)).sum()
# 查看cause_count
for k, v in cause_count.items():
    print(k, v)
# 生成词云
wc = WordCloud(font_path="./fonts/微软雅黑.ttf", background_color="white", width=1000, height=800, max_words=100, max_font_size=150)
wc.generate_from_frequencies(cause_count)
plt.figure(figsize=(10, 5), dpi=300)
# 保存
wc.to_file(f"./result/{keyword}_案由词云.png")

人格权纠纷 102
侵犯公民个人信息罪 52
隐私权纠纷 128
一般人格权纠纷 70
网络侵权责任纠纷 203
侵权责任纠纷 296
民事案由 49
商业贿赂不正当竞争纠纷 5
名誉权纠纷 391
不正当竞争纠纷 15
肖像权纠纷 22
姓名权纠纷 14
人事争议 1
侵害作品信息网络传播权纠纷 468
网络域名权属纠纷 1
航空旅客运输合同纠纷 3
网络购物合同纠纷 12
财产损害赔偿纠纷 15
其他行政管理 1
敲诈勒索罪 2
著作权权属、侵权纠纷 218
非法利用信息网络罪 2
合同纠纷 68
借记卡纠纷 5
非法获取公民个人信息罪 2
道路交通管理（道路） 1
买卖合同纠纷 3
劳动争议 3
信用卡纠纷 6
网络服务合同纠纷 12
行政复议 5
侵害作品复制权纠纷 1
民间借贷纠纷 3
商业诋毁纠纷 4
储蓄存款合同纠纷 6
侵害商标权纠纷 15
民政行政管理（民政） 1
行政处罚 4
侵犯著作权罪 3
网络域名权属、侵权纠纷 3
开设赌场罪 1
返还原物纠纷 2
治安管理（治安） 2
服务合同纠纷 26
电信服务合同纠纷 5
银行卡纠纷 6
骗取贷款、票据承兑、金融票证罪 1
小额借款合同纠纷 2
侵害发明专利权纠纷 2
假冒注册商标罪 1
工商行政管理（工商） 2
侵害出版者权纠纷 7
著作权权属纠纷 15
不当得利纠纷 1
保险纠纷 1
对下级法院执行异议裁定的复议 1
侵害作品署名权纠纷 2
城乡建设行政管理 1
侵害外观设计专利权纠纷 6
排除妨害纠纷 1
擅自使用他人企业名称、姓名纠纷 1
侵害商业秘密纠纷 3
诈骗罪 3
侵害其他著作财产权纠纷 1
行政补偿 1
提供劳务者受害责任纠纷 2
赌博罪 1
产品责任纠纷 3
行政行为 1
产品销售者责任纠纷 1
伪造、变造、买卖国家机关公文、证件、印章罪 1
侵害实用新型专利权纠纷 1
知识产权与竞争纠纷 2
出售、非法提供公民个人信息罪 1
金融借款合同纠纷 4
知识产权合同纠纷 1
侵害计算机软件著作权纠纷 1
身体权纠纷 1
商标权权属、侵权纠纷 2
机动车交通事故责任纠纷 3
行政案由 1
知识产权权属、侵权纠纷 10
分期付款买卖合同纠纷 1
确认合同无效纠纷 1
公证损害责任纠纷 1
侵害录音录像制作者权纠纷 4
案外人执行异议之诉 1
拍卖合同纠纷 1
著作权合同纠纷 2


In [24]:
# 对案由绘制横向的bar图，横坐标为数量，纵坐标为案由
# 先排序
cause_count = dict(sorted(cause_count.items(), key=lambda x: x[1], reverse=True)[:10])
sns.barplot(x=list(cause_count.values()), y=list(cause_count.keys()))
title = f"{keyword}_案由词频"
plt.xlabel("数量")
plt.ylabel("案由")
plt.title(title)
# 保存
plt.savefig(f"./result/{keyword}_案由词频.png", dpi=300, bbox_inches='tight')


<ipython-input-24-3ae9fd45e83a>:10: UserWarning: Glyph 32593 (\N{CJK UNIFIED IDEOGRAPH-7F51}) missing from current font.
  plt.savefig(f"./result/{keyword}_案由词频.png", dpi=300, bbox_inches='tight')
<ipython-input-24-3ae9fd45e83a>:10: UserWarning: Glyph 32476 (\N{CJK UNIFIED IDEOGRAPH-7EDC}) missing from current font.
  plt.savefig(f"./result/{keyword}_案由词频.png", dpi=300, bbox_inches='tight')
<ipython-input-24-3ae9fd45e83a>:10: UserWarning: Glyph 20010 (\N{CJK UNIFIED IDEOGRAPH-4E2A}) missing from current font.
  plt.savefig(f"./result/{keyword}_案由词频.png", dpi=300, bbox_inches='tight')
<ipython-input-24-3ae9fd45e83a>:10: UserWarning: Glyph 20154 (\N{CJK UNIFIED IDEOGRAPH-4EBA}) missing from current font.
  plt.savefig(f"./result/{keyword}_案由词频.png", dpi=300, bbox_inches='tight')
<ipython-input-24-3ae9fd45e83a>:10: UserWarning: Glyph 20449 (\N{CJK UNIFIED IDEOGRAPH-4FE1}) missing from current font.
  plt.savefig(f"./result/{keyword}_案由词频.png", dpi=300, bbox_inches='tight')
<ipython-input-

In [25]:
# 连接所有标签，做词频统计，保存词云
tags = []
for tag in df["标签"]:
    tags.extend(tag)
# 统计每个词出现的次数
tags_count = {}
for tag in tags:
    tags_count[tag] = tags.count(tag)
# 查看tags_count
for k, v in tags_count.items():
    print(k, v)
# 生成词云
wc = WordCloud(font_path="./fonts/微软雅黑.ttf", background_color="white", width=1000, height=800, max_words=100, max_font_size=150)
wc.generate_from_frequencies(tags_count)
plt.figure(figsize=(10, 5), dpi=300)
# 保存
wc.to_file(f"./result/{keyword}_标签词云.png")

# 对标签根据数量进行排序，绘制横向的bar图，横坐标为数量，纵坐标为标签，只绘制排名前10的
tags_count = sorted(tags_count.items(), key=lambda x: x[1], reverse=True)[:10]
tags_count = dict(tags_count)
sns.barplot(x=list(tags_count.values()), y=list(tags_count.keys()))
title = f"{keyword}_标签词频"
plt.xlabel("数量")
plt.ylabel("标签")
plt.title(title)
# 保存
plt.savefig(f"./result/{keyword}_标签词频.png", dpi=300, bbox_inches='tight')



个人隐私 328
网络服务 867
追加 116
迟延履行 801
连带责任 380
简易程序 607
备案 276
免责 94
申请人 110
法定义务 107
证据保全 349
公证书 1000
转为普通程序 49
名誉 633
票据 98
注意义务 440
普通程序 469
投诉 448
关联性 337
代理 1542
救济 65
违反法律 139
抗辩 408
加倍支付 886
委托 1532
缓刑 29
从轻处罚 57
取保候审 39
并处罚金 60
认罪 50
如实供述 47
证据不足 412
自首 14
立功 12
情节特别严重 38
从宽处理 12
工商登记 32
转载 317
新证据 174
法定程序 37
审判委员会 2
损害赔偿 490
损害事实 65
书面方式 18
网络侵权 294
知情权 49
当庭自愿认罪 5
查阅 47
复议 13
恶意诉讼 49
撤回上诉 254
监护人 13
精神损害 615
收购 8
共同侵权 112
监管 155
破产 5
物流 60
注销 140
破产清算 2
对账 25
营业执照 117
担保 51
告知义务 28
现场勘验 15
司法鉴定 45
延期审理 15
抗诉 6
减轻处罚 14
传唤 119
评估报告 4
网络游戏 14
依据不足 106
回收 7
公共利益 110
程序违法 40
不动产 5
劳动合同 62
社会保险 7
解除 79
投资 122
劳动关系 38
无效 62
劳动争议 33
退赃 12
初犯 11
利润 84
年费 6
专利 55
反诉 43
安全保障 36
注册资金 3
行政处罚 81
逾期 257
询问笔录 72
适格 258
物业管理 31
主体资格 103
中止审理 15
微信公众号 265
合并审理 35
管辖 14
设备所在地 24
国家秘密 13
拘留 13
民间借贷 11
恶意串通 4
汇款凭证 3
不开庭审理 21
商业秘密 48
怠于行使 5
适用法律错误 16
保险公司 13
通知义务 20
连带赔偿 40
安全标准 5
聘用 23
工资表 2
补充协议 54
赠送 72
恢复原状 23
分割 42
积极退赃 4
专属管辖 6
管辖权异议 62
合同履行地 10
书面协议 7
撤销权 1
违约金 15
辞职 29
管理人 75
变更登记 20
公司设立 14

<ipython-input-25-32c9a9c5eb3a>:28: UserWarning: Glyph 32593 (\N{CJK UNIFIED IDEOGRAPH-7F51}) missing from current font.
  plt.savefig(f"./result/{keyword}_标签词频.png", dpi=300, bbox_inches='tight')
<ipython-input-25-32c9a9c5eb3a>:28: UserWarning: Glyph 32476 (\N{CJK UNIFIED IDEOGRAPH-7EDC}) missing from current font.
  plt.savefig(f"./result/{keyword}_标签词频.png", dpi=300, bbox_inches='tight')
<ipython-input-25-32c9a9c5eb3a>:28: UserWarning: Glyph 20010 (\N{CJK UNIFIED IDEOGRAPH-4E2A}) missing from current font.
  plt.savefig(f"./result/{keyword}_标签词频.png", dpi=300, bbox_inches='tight')
<ipython-input-25-32c9a9c5eb3a>:28: UserWarning: Glyph 20154 (\N{CJK UNIFIED IDEOGRAPH-4EBA}) missing from current font.
  plt.savefig(f"./result/{keyword}_标签词频.png", dpi=300, bbox_inches='tight')
<ipython-input-25-32c9a9c5eb3a>:28: UserWarning: Glyph 20449 (\N{CJK UNIFIED IDEOGRAPH-4FE1}) missing from current font.
  plt.savefig(f"./result/{keyword}_标签词频.png", dpi=300, bbox_inches='tight')
<ipython-input-

In [26]:
# 使中文字体正常显示，用微软雅黑
plt.rcParams["font.sans-serif"] = ["Microsoft YaHei"]

# 对df通过日期排序
df.sort_values(by="日期", inplace=True)
# 统计每个年份的案件数量
year_count = df["日期"].apply(lambda x: x.year).value_counts()
# 查看year_count
for k, v in year_count.items():
    print(k, v)
# 绘制柱状图，并用一条曲线拟合,用sns
plt.figure(figsize=(10, 7), dpi=300)
sns.barplot(x=year_count.index, y=year_count.values)
plt.xticks(rotation=45)
plt.xlabel("年份")
plt.ylabel("案件数量")
plt.title(f"{keyword}案件数量随时间变化")
# 保存
plt.savefig(f"./result/{keyword}_案件数量随时间变化.png")

2020.0 534
2019.0 423
2021.0 295
2018.0 256
2017.0 145
2016.0 108
2015.0 102
2014.0 50
2013.0 34
2012.0 16
2011.0 13
2008.0 8
2009.0 6
2010.0 5
2006.0 1


In [33]:
df["法院"].value_counts()

北京互联网法院          226
北京市海淀区人民法院       117
重庆自由贸易试验区人民法院    109
北京知识产权法院          93
广东省深圳市罗湖区人民法院     51
                ... 
佛山市禅城区人民法院         1
成都市金牛区人民法院         1
乌海市海勃湾区人民法院        1
南京市雨花台区人民法院        1
沂水县人民法院            1
Name: 法院, Length: 568, dtype: int64

In [28]:
# 提取'法院'里面的省份
provinces = df["法院"].apply(lambda x: x[:2])
# provinces转成一个数组
provinces = provinces.values
len(provinces)

2000

In [29]:
province_city_dic = {
    '北京': ['北京'],
    '上海': ['上海'],
    '深圳': ['深圳'],
    '天津': ['天津'],
    '重庆': ['重庆'],
    '澳门': ['澳门'],
    '香港': ['香港'],
    '海南': ['海口', '三亚'],
    '台湾': ['台湾', '台北', '高雄', '基隆', '台中', '台南', '新竹', '嘉义'],
    '河北': ['唐山', '邯郸', '邢台', '保定', '承德', '沧州', '廊坊', '衡水', '石家庄', '秦皇岛', '张家口'],
    '山西': ['太原', '大同', '阳泉', '长治', '晋城', '朔州', '晋中', '运城', '忻州', '临汾', '吕梁'],
    '山东': ['济南', '青岛', '淄博', '枣庄', '东营', '烟台', '潍坊', '济宁', '泰安', '威海', '日照', '莱芜', '临沂', '德州', '聊城', '滨州', '荷泽', '菏泽'],
    '江苏': ['南京', '无锡', '徐州', '常州', '苏州', '南通', '淮安', '盐城', '扬州', '镇江', '泰州', '宿迁', '连云港'],
    '浙江': ['杭州', '宁波', '温州', '嘉兴', '湖州', '绍兴', '金华', '衢州', '舟山', '台州', '丽水'],
    '安徽': ['合肥', '芜湖', '蚌埠', '淮南', '淮北', '铜陵', '安庆', '黄山', '滁州', '阜阳', '宿州', '巢湖', '六安', '亳州', '池州', '宣城', '马鞍山'],
    '福建': ['福州', '厦门', '莆田', '三明', '泉州', '漳州', '南平', '龙岩', '宁德'],
    '江西': ['南昌', '萍乡', '新余', '九江', '鹰潭', '赣州', '吉安', '宜春', '抚州', '上饶', '景德镇'],
    '河南': ['郑州', '开封', '洛阳', '焦作', '鹤壁', '新乡', '安阳', '濮阳', '许昌', '漯河', '南阳', '商丘', '信阳', '周口', '驻马店', '济源', '平顶山', '三门峡'],
    '湖北': ['武汉', '黄石', '襄樊', '十堰', '荆州', '宜昌', '荆门', '鄂州', '孝感', '黄冈', '咸宁', '随州', '恩施', '仙桃', '天门', '潜江'],
    '湖南': ['长沙', '株洲', '湘潭', '衡阳', '邵阳', '岳阳', '常德', '益阳', '郴州', '永州', '怀化', '娄底', '吉首', '张家界'],
    '广东': ['广州', '深圳', '珠海', '汕头', '韶关', '佛山', '江门', '湛江', '茂名', '肇庆', '惠州', '梅州', '汕尾', '河源', '阳江', '清远', '东莞', '中山', '潮州', '揭阳', '云浮'],
    '广西': ['南宁', '柳州', '桂林', '梧州', '北海', '钦州', '贵港', '玉林', '百色', '贺州', '河池', '来宾', '崇左', '防城港'],
    '四川': ['成都', '自贡', '泸州', '德阳', '绵阳', '广元', '遂宁', '内江', '乐山', '南充', '宜宾', '广安', '达州', '眉山', '雅安', '巴中', '资阳', '西昌', '攀枝花'],
    '贵州': ['贵阳', '遵义', '安顺', '铜仁', '毕节', '兴义', '凯里', '都匀', '六盘水', '黔西南布依族苗族自治州', '黔东南苗族侗族自治州', '黔南布依族苗族自治州'],
    '云南': ['昆明', '曲靖', '玉溪', '保山', '昭通', '丽江', '思茅', '临沧', '景洪', '楚雄', '大理', '潞西'],
    '陕西': ['西安', '铜川', '宝鸡', '咸阳', '渭南', '延安', '汉中', '榆林', '安康', '商洛'],
    '甘肃': ['兰州', '金昌', '白银', '天水', '武威', '张掖', '平凉', '酒泉', '庆阳', '定西', '陇南', '临夏', '合作', '嘉峪关'],
    '辽宁': ['沈阳', '大连', '鞍山', '抚顺', '本溪', '丹东', '锦州', '营口', '盘锦', '阜新', '辽阳', '铁岭', '朝阳', '葫芦岛'],
    '吉林': ['长春', '吉林', '四平', '辽源', '通化', '白山', '松原', '白城', '延吉'],
    '黑龙江': ['鹤岗', '鸡西', '大庆', '伊春', '黑河', '绥化', '双鸭山', '牡丹江', '佳木斯', '七台河', '哈尔滨', '齐齐哈尔'],
    '青海': ['西宁', '德令哈', '格尔木'],
    '宁夏': ['银川', '吴忠', '固原', '中卫', '石嘴山'],
    '西藏': ['拉萨', '日喀则'],
    '新疆': ['哈密', '和田', '喀什', '昌吉', '博乐', '伊宁', '塔城', '吐鲁番', '阿图什', '库尔勒', '五家渠', '阿克苏', '阿勒泰', '石河子', '阿拉尔', '乌鲁木齐', '克拉玛依', '图木舒克'],
    '内蒙古': ['包头', '乌海', '赤峰', '通辽', '鄂尔多斯', '呼伦贝尔', '巴彦淖尔', '乌兰察布', '兴安盟', '呼和浩特', '锡林郭勒盟', '阿拉善盟', '巴彦淖尔盟', '乌兰察布盟']
}
# 根据province_city_dic,把province里面的一些城市改成对应的省份
for idx, pro in enumerate(provinces):
    for pro_, cities in province_city_dic.items():
        if pro in cities:
            provinces[idx] = pro_
            # print(f"城市{pro}在{pro_}省")
            break

print(provinces)

['广东' '重庆' '上海' ... '天津' '北京' '北京']


In [30]:
# 对 provinces做词频统计，并在地图上标注出数量
provinces_count = {}
for pro in provinces:
    if pro not in provinces_count:
        provinces_count[pro] = 0
    provinces_count[pro] += 1



In [31]:
# ! pip install echarts-countries-pypkg #世界地图
# ! pip install echarts-china-provinces-pypkg #中国省份
# ! pip install echarts-china-cities-pypkg #中国城市
# ! pip install echarts-china-counties-pypkg #中国县区
# ! pip install echarts-china-misc-pypkg #中国区域

In [32]:
# -*- coding: utf-8 -*-
# from pyecharts.faker import Faker
# from pyecharts import options as opts
# from pyecharts.charts import Map

# customMap = (
#     Map().add("商家A",
#              [list(z) for z in zip(provinces_count.keys(), provinces_count.values())],  #数据
#              "china"  #地图
#              )
#         .set_global_opts(  #全局项
#         title_opts=opts.TitleOpts(  #标题项
#             title="中国地图"  #标题名称
#         )
#     )
# )

# customMap.render("map_test.html")  # 生成名为map_test的本地html文件
